In [2]:
!pip install ccxt pandas scikit-learn nltk pytz mysql-connector-python

In [9]:
import ccxt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from datetime import datetime
import pytz
import mysql.connector
from mysql.connector import Error

nltk.download('vader_lexicon')
sia = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/mjb/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [10]:
#con ccxt inicializo la api de Binance
exchange = ccxt.binance()

# elijo el simbolo o ticker de bitcoin y la franja de tiempo
symbol = 'BTC/USDT'
timeframe = '1h'

# aca voy a obtener los milisegundos desde la fecha que elija para entrenar el futuro arbol de decision
since = exchange.parse8601('2021–01–01T00:00:00Z')

# este va a ser la cantidad de puntos o "velas" que vamos a recuperar
limit = 1000 

# obtenemos las (open-high-low-close-volume bien resumido jeje) OHLCV, de bitcoin, en el marco de tiempo de 1h, desde 2021, con un limite de solo 1000 "velas"
# y luego le asignamos columnas y convertimos todos los datos obtenidos a un dataframe
ohlcv = exchange.fetch_ohlcv(symbol, timeframe, since, limit)
columns = ['timestamp', 'open', 'high', 'low', 'close', 'volume']
data = pd.DataFrame(ohlcv, columns=columns)

# convertimos los milisegundos de unix en un formato de tipo fecha, para poder analizarlo como datetime en un arbol de decision
data['timestamp'] = pd.to_datetime(data['timestamp'], unit='ms')
print(data.head())

            timestamp      open      high       low     close      volume
0 2024-04-25 06:00:00  64223.36  64391.99  64195.70  64288.00   363.60804
1 2024-04-25 07:00:00  64288.00  64379.85  63824.07  64017.99   951.17474
2 2024-04-25 08:00:00  64017.99  64176.86  63708.20  63948.78  1329.27338
3 2024-04-25 09:00:00  63948.78  64192.00  63809.13  63966.02   833.85634
4 2024-04-25 10:00:00  63966.03  63966.03  63337.25  63462.01  2418.87397


In [12]:
conn = None
try:
    conn = mysql.connector.connect(
        host='localhost',
        database='backtesting',
        user='pruebas',
        password='contraseñaprueba'
    )
    if conn.is_connected():
        cursor = conn.cursor()

        # Crea la tabla si no existe
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS btc_usdt (
                timestamp DATETIME,
                open DOUBLE,
                high DOUBLE,
                low DOUBLE,
                close DOUBLE,
                volume DOUBLE,
                PRIMARY KEY (timestamp)
            )
        ''')

        # Inserta los datos en la tabla
        for row in data.itertuples(index=False):
            cursor.execute('''
                REPLACE INTO btc_usdt(timestamp, open, high, low, close, volume)
                VALUES (%s, %s, %s, %s, %s, %s)
            ''', (
                row.timestamp.strftime('%Y-%m-%d %H:%M:%S'),
                row.open,
                row.high,
                row.low,
                row.close,
                row.volume
            ))

        # Confirma los cambios
        conn.commit()
        print("Datos almacenados en la base de datos.")

except Error as e:
    print(f"Error: {e}")
finally:
    if conn is not None and conn.is_connected():
        cursor.close()
        conn.close()
        print("Conexión a MySQL cerrada.")

Datos almacenados en la base de datos.
Conexión a MySQL cerrada.


In [5]:
# IMPORTANTE: los primeros 10 registros de la media movil van a ser nulos debido a que aun no hay lo suficientes datos para calcular esta misma
data['MA_10'] = data['close'].rolling(window=10).mean()

# estas lineas se utilizaran cuando se carguen noticias al DF para realizar el analisis de sentimiento
# data['Sentiment'] = data['Noticias'].apply(lambda x: sia.polarity_scores(x)['compound'])

print(data.head())

            timestamp      open      high       low     close      volume  \
0 2024-04-22 09:00:00  65961.58  66055.17  65801.76  65938.00   798.46267   
1 2024-04-22 10:00:00  65938.00  66154.87  65890.00  66048.18   793.68807   
2 2024-04-22 11:00:00  66048.18  66108.00  65702.00  65877.47   899.90178   
3 2024-04-22 12:00:00  65877.47  65995.48  65626.87  65995.48  1032.65578   
4 2024-04-22 13:00:00  65995.48  66260.54  65693.98  66114.00  1532.55331   

   MA_10  
0    NaN  
1    NaN  
2    NaN  
3    NaN  
4    NaN  


In [6]:
#con Shift desplazamos el indice actual al anterior, con la idea de comparar la columna close con su anterior y en caso de ser mayor devuelva 1 o 0 (True or False)
data['Next_Event'] = (data['close'].shift(-1) > data['close']).astype(int)
#este va a ser el objetivo, el cual el algoritmo debe predecir
target = 'Next_Event'

#ahora si eliminamos los valores nulos de la MA10, para el entrenamiento
data = data.dropna()
#filtramos las columnas 
features = ['MA_10', 'volume', 'open', 'high', 'low', 'close']
X = data[features]
#y las separamos de la variable objetivo
y = data[target]

#realizamos la division de los conjuntos de entrenamiento y los conjuntos de pruebas
#test_size->0.2 = 20%, este 20% del total de datos seran las pruebas
#random_state=42 se asegura que no varie las divisiones realizadas en el entrenamiento
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


model = DecisionTreeClassifier(random_state=42)
model.fit(X_train, y_train)
# Evaluate the model
accuracy = model.score(X_test, y_test)
print(f'Model Accuracy: {accuracy:.2f}')

Model Accuracy: 0.56


In [7]:
zona_horaria = pytz.timezone('America/Argentina/Buenos_Aires')
now = datetime.now(zona_horaria)
formato_now = now.strftime("%Y-%m-%d %H:%M:%S")

#una vez entrenado el modelo, probamos chequeando con los ultimos datos una posible prediccion de si el valor de bitcoin sube o baja
new_data = pd.DataFrame({
 'MA_10': [data['MA_10'].iloc[-1]],
 'volume': [data['volume'].iloc[-1]],
 'open': [data['open'].iloc[-1]],
 'high': [data['high'].iloc[-1]],
 'low': [data['low'].iloc[-1]],
 'close': [data['close'].iloc[-1]]
})

next_event = model.predict(new_data)

#agregamos tambien la fecha y hora actual con la idea de mas adelante ir guardando estas predicciones y compararlas con el precio de BTC
print(formato_now)
print(f'Predicted Next Event: {"Price Increase" if next_event[0] == 1 else "Price Decrease"}')


2024-06-02 21:00:20
Predicted Next Event: Price Decrease
